In [17]:
# pip install playwright

In [52]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

In [53]:
async def scrape_jobstreet():
    scrape_start_time = datetime.now()

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://ph.jobstreet.com/Data-Science-jobs?sortmode=ListedDate")

        print("🔍 Waiting for job listings to load...")
        await page.wait_for_selector("article[data-testid='job-card']")
        await asyncio.sleep(5)  # Ensure full DOM is loaded

        jobs = await page.query_selector_all("article[data-testid='job-card']")
        print(f"📄 Found {len(jobs)} job listings")

        data = []

        for i, job in enumerate(jobs):
            try:
                print(f"👉 Clicking job {i+1}...")
                await job.click()
                await asyncio.sleep(2)  # Let right panel populate

                # Wait for key detail to appear
                await page.wait_for_selector("div[data-automation='jobAdDetails']", timeout=5000)

                # Extract job details
                job_title_el = await page.query_selector("h1[data-automation='job-detail-title']")
                job_title = await job_title_el.inner_text() if job_title_el else "N/A"

                company_el = await page.query_selector("span[data-automation='advertiser-name']")
                company = await company_el.inner_text() if company_el else "N/A"

                location_el = await page.query_selector("span[data-automation='job-detail-location']")
                location = await location_el.inner_text() if location_el else "N/A"

                work_type_el = await page.query_selector("span[data-automation='job-detail-work-type']")
                work_type = await work_type_el.inner_text() if work_type_el else "N/A"

                classification_el = await page.query_selector("span[data-automation='job-detail-classifications']")
                classification = await classification_el.inner_text() if classification_el else "N/A"

                # Salary: check which one exists
                salary_el = await page.query_selector("span[data-automation='job-detail-salary']")
                if salary_el:
                    salary = await salary_el.inner_text()
                else:
                    salary = ""

                # Description block
                desc_el = await page.query_selector("div[data-automation='jobAdDetails']")
                job_description = await desc_el.inner_text() if desc_el else "N/A"

                # Job URL from job card element
                anchor = await job.query_selector("a[data-automation='job-list-view-job-link']")
                url_suffix = await anchor.get_attribute("href") if anchor else ""
                job_url = "https://ph.jobstreet.com" + url_suffix if url_suffix else "N/A"

                # Posted time from job card
                posted_el = await job.query_selector("span[data-automation='jobListingDate']")
                posted_raw = await posted_el.inner_text() if posted_el else ""
                posted_datetime = "N/A"
                try:
                    if "m" in posted_raw:
                        minutes = int(posted_raw.split("m")[0].strip())
                        posted_time = scrape_start_time - timedelta(minutes=minutes)
                    elif "hr" in posted_raw:
                        hours = int(posted_raw.split("hr")[0].strip())
                        posted_time = scrape_start_time - timedelta(hours=hours)
                    elif "d" in posted_raw:
                        days = int(posted_raw.split("d")[0].strip())
                        posted_time = scrape_start_time - timedelta(days=days)
                    else:
                        posted_time = scrape_start_time
                    posted_datetime = posted_time.strftime("%d/%m/%y %H:%M:%S")
                except Exception as e:
                    print(f"⚠️ Failed to parse posted time '{posted_raw}': {e}")

                data.append({
                    "Job Title": job_title,
                    "Company": company,
                    "Location": location,
                    "Work Type": work_type,
                    "Classification": classification,
                    "Salary": salary,
                    "Job Description": job_description,
                    "Job URL": job_url,
                    "Posted Time": posted_datetime
                })

            except Exception as e:
                print(f"⚠️ Error on job {i+1}: {e}")
                continue

        await browser.close()
        return pd.DataFrame(data)

In [54]:
# Run the async function inside Jupyter
df = await scrape_jobstreet()
df.head()

🔍 Waiting for job listings to load...
📄 Found 32 job listings
👉 Clicking job 1...
👉 Clicking job 2...
👉 Clicking job 3...
👉 Clicking job 4...
👉 Clicking job 5...
👉 Clicking job 6...
👉 Clicking job 7...
👉 Clicking job 8...
👉 Clicking job 9...
👉 Clicking job 10...
👉 Clicking job 11...
👉 Clicking job 12...
👉 Clicking job 13...
👉 Clicking job 14...
👉 Clicking job 15...
👉 Clicking job 16...
👉 Clicking job 17...
👉 Clicking job 18...
👉 Clicking job 19...
👉 Clicking job 20...
👉 Clicking job 21...
👉 Clicking job 22...
👉 Clicking job 23...
👉 Clicking job 24...
👉 Clicking job 25...
👉 Clicking job 26...
👉 Clicking job 27...
👉 Clicking job 28...
👉 Clicking job 29...
👉 Clicking job 30...
👉 Clicking job 31...
👉 Clicking job 32...


,Job Title,Company,Location,Work Type,Classification,Salary,Job Description,Job URL
0,Market Analyst,Moldex Group of Companies,"Quezon City, Metro Manila",Full time,Market Research & Analysis (Marketing & Commun...,,Duties and Responsibilities:\n\nMarket Researc...,https://ph.jobstreet.com/job/85851432?type=sta...
1,IT Technician,OFFSHORE BUSINESS PROCESSING INC.,"Mandaluyong City, Metro Manila",Full time,Help Desk & IT Support (Information & Communic...,"₱32,000 – ₱36,000 per month",Job Summary\n\nHMO on Day 1\n\nReceive promisi...,https://ph.jobstreet.com/job/85851256?type=sta...
2,Senior Business Analyst,"Trends Group, Inc.","Makati City, Metro Manila",Full time,Business/Systems Analysts (Information & Commu...,,Duties and Responsibilities:\n\nGather and doc...,https://ph.jobstreet.com/job/85851064?type=sta...
3,Java Developer - Spring App Framework | Up to ...,Accenture,Metro Manila,Full time,Developers/Programmers (Information & Communic...,,Ready to become part of the dynamic Accenture ...,https://ph.jobstreet.com/job/85850461?type=sta...
4,Programmer (Java - Spring Boot) | Up to PHP 80...,Accenture,Metro Manila,Full time,Developers/Programmers (Information & Communic...,,Ready to become part of the dynamic Accenture ...,https://ph.jobstreet.com/job/85850460?type=sta...


# Testing Space